In [68]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
import matplotlib.pyplot as plt
dir_path='/Users/hudsonyeo/Desktop/Python/leo/data/day'
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day')
import time
def process_dir(dir_list):
    x=[]
    for dir_ in dir_list:
        if dir_!='.DS_Store':
            x.append(dir_path+'/'+dir_+'/')
    return x
standard_columns=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/2019.01.02.csv').columns

In [69]:
dir_list=os.listdir(dir_path)
dir_list.sort()
dir_list=process_dir(dir_list)
dir_list

['/Users/hudsonyeo/Desktop/Python/leo/data/day/RM/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/SR/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/c/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/fu/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/m/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/ni/',
 '/Users/hudsonyeo/Desktop/Python/leo/data/day/zn/']

In [13]:
def get_file_list(dir_path):
    return os.listdir(dir_path)

class categorise():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(5)
                elif i<self.threshold[1][1]:
                    temp.append(6)
                elif i<self.threshold[1][2]:
                    temp.append(7)
                else:
                    temp.append(8)
            if i<0:
                if i>self.threshold[0][2]:
                    temp.append(4)
                elif i>self.threshold[0][1]:
                    temp.append(3)
                elif i>self.threshold[0][0]:
                    temp.append(2)
                else:
                    temp.append(1)
        return np.asarray(temp)
    
class categorise_simple():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[25,50,75]
        
    def fit(self,array):
        self.threshold.append(np.percentile(array,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                else:
                    temp.append(4)
        return np.asarray(temp)    
    
class categorise_10():
    def __init__(self):
        self.threshold=[]
        self.percentiles=[10,20,30,40,50,60,70,80,90]
        
    def fit(self,array):
        positive=array[array>0]
        negative=array[array<0]
        self.threshold.append(np.percentile(negative,self.percentiles))   
        self.threshold.append(np.percentile(positive,self.percentiles))
     
    def return_quartile(self,array):
        temp=[]
        for i in array:
            if i>=0:
                if i<self.threshold[1][0]:
                    temp.append(11)
                elif i<self.threshold[1][1]:
                    temp.append(12)
                elif i<self.threshold[1][2]:
                    temp.append(13)
                elif i<self.threshold[1][3]:
                    temp.append(14)
                elif i<self.threshold[1][4]:
                    temp.append(15)
                elif i<self.threshold[1][5]:
                    temp.append(16)
                elif i<self.threshold[1][6]:
                    temp.append(17)
                elif i<self.threshold[1][7]:
                    temp.append(18)
                elif i<self.threshold[1][8]:
                    temp.append(19)                    
                else:
                    temp.append(20)
            if i<0:
                if i<self.threshold[0][0]:
                    temp.append(1)
                elif i<self.threshold[0][1]:
                    temp.append(2)
                elif i<self.threshold[0][2]:
                    temp.append(3)
                elif i<self.threshold[0][3]:
                    temp.append(4)
                elif i<self.threshold[0][4]:
                    temp.append(5)
                elif i<self.threshold[0][5]:
                    temp.append(6)
                elif i<self.threshold[0][6]:
                    temp.append(7)
                elif i<self.threshold[0][7]:
                    temp.append(8)
                elif i<self.threshold[0][8]:
                    temp.append(9)                    
                else:
                    temp.append(10)
        return np.asarray(temp)    

In [14]:
def calc_smart_price(dataset):
    data=dataset[:]
    
    #to combat the limit up event, where price is set to 0. 
    rows=(data.loc[:,'BidPrice1']==0) #count rows of bid price equal 0
    if (np.any(rows)): #if there is such a row
        data.at[rows,'BidPrice1']=data.loc[rows,'AskPrice1'] #for that row, assign ask price to it
    rows=(data.loc[:,'AskPrice1']==0) #do the same for ask price
    if (np.any(rows)):
        data.at[rows,'AskPrice1']=data.loc[rows,'BidPrice1'] 
        
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_present_vol(dataset):
    data=dataset[:]
    data['current_vol']=data.loc[:,'Volume'].diff().fillna(0)/2
    return data

def calc_future_price(dataset,time_ahead=30):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,44]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),44]):
            j+=1
        #print(i,j,(data[(i+j-1),44]))
        if (i+j)<length:
            future_price.append(data[(i+j),51]) #51 is the index for smart price            
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset):
    data=dataset.copy()
    temp=data[:,53]-data[:,51]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset):
    data=dataset[:]
    index=data[:,44]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,44]=new_index
    return data

def calc_sma_fast(dataset,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,51])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)

    sma_values=np.asarray(sma_values)
    sma_values=data[:,51]-sma_values
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)  

def calc_past_vol(dataset,duration=1): #
    data=dataset[:]
    vol_values=[] 
    vol_sum=np.cumsum(data[:,52])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            vol=(vol_sum[i]-vol_sum[i-j])
            vol_values.append(vol)
        else:
            vol=vol_sum[i]
            vol_values.append(vol)
    vol_values=np.asarray(vol_values)
    vol_values=np.expand_dims(vol_values,axis=1)
    return np.concatenate((data,vol_values),axis=1) #52  

def process(dataset,sma_duration=1,vol_duration=1):
    data=dataset[:]
    data=calc_smart_price(data) #51
    data=calc_present_vol(data).values #52
    data=set_index(data)
    data=calc_future_price(data) #53
    data=calc_edge(data) #54
    data=calc_sma_fast(data,duration=sma_duration) #55
    data=calc_past_vol(data,duration=vol_duration) #56
    return data

In [15]:
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
def process_files(path,file_list,sma_duration=1):
    df_list=[]
    name_list=[]
    print('processing files...')
    for file in file_list: #read all files and add them to file_list
        if file[-3:]=='csv': #check if file is a CSV
            name_list.append(file)
            df_list.append(process(pd.read_csv(path+file),sma_duration=sma_duration))   
    print('complete processing')
    return name_list,df_list

In [16]:
def run_analysis(df_path, df_list,name_list,sma_duration=1):
    final_df=pd.DataFrame()    

    for i in range(len(df_list)): #for each 20 day rolling window
        if i<19:
            continue
        if i==40:
            print('50% complete')
        
        
        #get -19 day
        sma=df_list[i-19][:,55].copy() #column for SMA
        vol=df_list[i-19][:,56].copy()

        #get -18 to 0 day (19 days in total)
        for k in range((i-18),i+1): #get 20 day moving averages
            sma=np.concatenate((sma,df_list[k][:,55].copy()))
            vol=np.concatenate((vol,df_list[k][:,56].copy()))

        cat_sma=categorise_10()
        cat_sma.fit(sma) #calculate quartile thresholds for past 20 days

        #get categories for past 20 days
        cat_sma_20=(cat_sma.return_quartile(sma))   

        #get x,y for regression
        x_today=df_list[i][:,55].copy().astype(float) #column for SMA     
        y_today=df_list[i][:,54].copy().astype(float) #column for edge

        vol_today=df_list[i][:,56].copy()    
        #removing all NA
        isnum=(~np.isnan(x_today)) & (~np.isnan(y_today))
        #get categories of today's sma  
        cat_x_today=cat_sma.return_quartile(x_today)


        reg_result={}
        reg_result['date']=name_list[i]
        reg_result['total_obs']=len(x_today[isnum])   

        for cat in range(1,21):
            #check past 20 days quartiles and filter volume by category
            vol_fit=vol[cat_sma_20==cat].copy()

            #simple quartile categorisation
            cat_vol=categorise_simple()
            cat_vol.fit(vol_fit)


            #today's sma filter
            sma_filter_today=(cat_x_today==cat)
            #today's vol categories
            cat_vol_today=cat_vol.return_quartile(vol_today)


            for quartile in [1,2,3,4]:

                filtered= (isnum & sma_filter_today) #filtering NA and sma quartile
                filtered= filtered & (cat_vol_today==quartile) #filtering volume
                new_y=y_today[filtered].copy()

                q='ma_cat_'+str(cat)+'_vol_quartile'+str(quartile)+'_'
                if (len(new_y)!=0):

                    reg_result[(q+'mean')]=np.mean(new_y)
                    reg_result[(q+'std')]=np.std(new_y)
                    reg_result[(q+'num_obs')]=len(new_y)

                else:

                    reg_result[(q+'mean')]=np.nan
                    reg_result[(q+'std')]=np.nan
                    reg_result[(q+'num_obs')]='0'

        final_df=final_df.append(reg_result,ignore_index=True)                
    temp=df_path+'result_10sma,6vol_1min,'+str(sma_duration)+'min_sma,vol_mean.csv'
    final_df.to_csv(temp)
    print('done',temp)   

In [ ]:

for dir_path in dir_list:
    file_list=get_file_list(dir_path)
    for sma_duration in [1,3,5,10]:
        start=time.time()
        print('start',dir_path,sma_duration) 
        #check if columns are exactly the same as TA
        test_columns=pd.read_csv(dir_path+file_list[0]).columns 
        #if same 
        if (len(standard_columns)==len(test_columns)) and (np.all(test_columns==standard_columns)): 
            name_list,df_list=process_files(dir_path, file_list,sma_duration=sma_duration) #process, get SMA and vol data
            mid=time.time()
            print(mid-start)
            #split into categories and get means
            run_analysis(dir_path+'/results/', df_list,name_list,sma_duration=sma_duration)  
            end=time.time()
            print('complete',end-mid)
        else:
            print('columns do not match!')
            print('skipping this folder')


In [66]:
data=pd.read_csv('/Users/hudsonyeo/Desktop/Python/leo/data/day/SR/results/result_10sma,6vol_1min,10min_sma,vol_mean.csv')
x=[]
x_std=[]
pos=[]
num_obs=[]
for ma_cat in range(1,21):
    for vol_cat in range(1,5):
        col='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_mean'
        std=data.loc[:,col].std()
        mean=data.loc[:,col].mean()
        column=data.loc[:,col]
        x_std.append(std)
        x.append(mean)
        percentage=len(column[column>=0])/len(column)
        pos.append(percentage)  
        
        obs='ma_cat_'+str(ma_cat)+'_vol_quartile'+str(vol_cat)+'_num_obs'
        obs=data.loc[:,obs].sum()
        num_obs.append(obs)
x=np.asarray(x)
x=np.reshape(x,(20,4))
x=np.around(x,3)
x_std=np.asarray(x_std)
x_std=np.reshape(x_std,(20,4))
pos=np.asarray(pos)
pos=np.reshape(pos,(20,4))
num_obs=np.asarray(num_obs)
num_obs=np.reshape(num_obs,(20,4))
print('mean\n',x)
print('std\n',x_std)
print('% positive\n',pos)
print('num obs\n',num_obs)

mean
 [[ 0.448  0.577  0.436  0.53 ]
 [ 0.208  0.144  0.085  0.108]
 [ 0.189  0.005  0.119 -0.069]
 [ 0.005  0.141 -0.117  0.057]
 [ 0.029  0.003 -0.064  0.119]
 [ 0.018 -0.009 -0.137  0.029]
 [-0.107  0.023 -0.067  0.056]
 [-0.104 -0.009 -0.043 -0.06 ]
 [ 0.075  0.091 -0.059 -0.038]
 [ 0.398 -0.002 -0.073  0.064]
 [ 0.155  0.016  0.005  0.089]
 [ 0.045 -0.032  0.029  0.033]
 [ 0.101  0.038  0.069 -0.078]
 [ 0.12  -0.02   0.066 -0.136]
 [ 0.069 -0.014  0.074  0.039]
 [-0.153  0.076  0.118 -0.088]
 [-0.007  0.065  0.018 -0.085]
 [-0.12   0.043  0.086  0.055]
 [-0.243 -0.136  0.148  0.223]
 [-0.414 -0.179 -0.158  0.03 ]]
std
 [[1.26746542 1.19512975 1.12939333 1.74521779]
 [0.68303335 0.8916933  0.77864566 1.47492986]
 [0.54312903 0.49062746 0.63812228 1.16360265]
 [0.5234092  0.44288723 0.66778554 0.69527167]
 [0.39508051 0.44341839 0.40547453 0.83398717]
 [0.79217404 0.44680267 0.42598712 0.63064112]
 [0.75249204 0.38836115 0.36147737 0.51747711]
 [0.48515669 0.33226883 0.36644327 0.66